In [19]:
"""
Authors: Co[nn]action Team - Eren Godze Anil, Jun Wen Loo, Roy Uijtendaal

ADJACENCIES & MODULE TYPES
This script is Part C in Configuration workflow. The aim is to pull necessary information from a graph (nodes, edges) such as start and end nodes of an edge 
to compare these and determine the connection on that edge (open, closed or closed and buttressed)

After the open/closed data is put into the nodes' nested dictionary to assign module types to the nodes.
"""

"\nAuthors: Co[nn]action Team - Eren Gozde Anil, Jun Wen Loo, Roy Uijtendaal\n\nADJACENCIES & MODULE TYPES\nThis script is Part C in Configuration workflow. The aim is to pull necessary information from a graph (nodes, edges) such as start and end nodes of an edge \nto compare these and determine the connection on that edge (open, closed or closed and buttressed)\n\nAfter the open/closed data is put into the nodes' nested dictionary to assign module types to the nodes.\n"

In [20]:
"""Step 0: Import necessary libraries"""

#INITIALISATION
import networkx as nx
import itertools
import json 
import re, requests

In [21]:
# DEFINITIONS:

# Gives adjacency dictionary
edge_dict={}
def create_edge_dict(graph):
    for i, n in G.adjacency():
        # print("i is",i)
        # print("n is",n)
        edge_dict[i] = list(n)
    return edge_dict

# Gives the parent of the value in a nested dictionary
def find_key(d, value):
    start_list =[]
    for i in value:
        # print("i is", i)
        for k,v in d.items():
            # print("k and v is", k,v)
            if v['name'] == i:
                # print("k is", k)
                start_list.append(i)
    return start_list

# Gives the value of tag key of a node
def retrieve_tag(list):
    tag_list = []
    for i in list:
        # print ("i is", i)
        tag = nodes[i]['tag']
        # print(tag)
        tag_list.append(tag)
    return (tag_list)

# Creates unique combinations from given string of available combinations
def create_combinations(combination_str):
    permutation_list = list(set(itertools.permutations(str(combination_str), 4)))
    p_list = []
    for i in range(len(permutation_list)):
            a = [int(x) for x in permutation_list[i]]
            p_list.append(a)
    return p_list

In [22]:
"""Step 1: Import necessary data from Github"""

# Import the nodes data from github
# Import from online repository
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Allocation/211024_nodes_occupied.txt"
resp = requests.get(url)
nodes = json.loads(resp.text)

# Converts keys from str to int
nodes = {int(k) : v for k,v in nodes.items()}

# Import edges data from github
# Import from online repository
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Foundation/edges.txt"
resp = requests.get(url)
edges = json.loads(resp.text)

#Converts nested lists into a list of tuples
edges = [tuple(i) for i in edges]

#Number of nodes in a level
nodes_count_1 = 580

In [23]:
"""Step 2: Initiate Graph with the given edges and nodes"""

# It should be noted that we only draw the graph to visualise the connections 
# and to use some fuctions that can only be used with graphs for the sake of efficiency
# There is no position of the nodes
G = nx.Graph()
G.add_edges_from(edges)

# Get adjacency dictionary
edge_dict = create_edge_dict(G)

In [24]:
"""Step 3: Assess Adjacencies"""

# Create a dictionary of open/closed lists attributed to each node
edge_dict_connection={}

# Determine the connection conditions based on the following criteria ordered by priority 
# The condition that 'tags are different' is at the end because if the tag is x or None or a street tag, the tags are still different
for k,v in edge_dict.items():
    edge_connection_list=[]
    for i in v:
        # If two neighboring nodes have the same tag then the connection is open(0)
        if nodes[k]["tag"]==nodes[i]["tag"]:
            same_shop = 0
            edge_connection_list.append(same_shop)
        # If one of the nodes is a street node(local street 0, middle street 1, main street 2) then the connection is open(0)
        elif (nodes[k]["tag"]==0) or (nodes[i]["tag"]==0) or (nodes[k]["tag"]==1) or (nodes[i]["tag"]==1) or (nodes[k]["tag"]==2) or (nodes[i]["tag"]==4 or (nodes[k]["tag"]==4)):
            street_shop = 0
            edge_connection_list.append(street_shop)
        # If one of the nodes is out of the set boundary (x) or if there is no shop on it(None) then the connection is closed and there will be a buttress(2) 
        elif (nodes[k]["tag"]=="x") or (nodes[i]["tag"]=="x") or (nodes[k]["tag"]==None) or (nodes[i]["tag"]==None):
            nothing = 2
            edge_connection_list.append(nothing)
        # In case of any other condition such as tags being different(shops owned by different people) then the connection is closed(1)
        else:
            edge_oc_closed = (1)
            edge_connection_list.append(edge_oc_closed)
        edge_dict_connection[k]=edge_connection_list


In [25]:
"""Step 4: Introduce a new attribute "connection" to nodes dictionary"""

# Adds the connection data (eg:[0,0,0,0]) to nodes dictionary
for k in nodes:
    nodes[k]["connection"]=None
for k,v in edge_dict_connection.items():
    nodes[k]["connection"]=[]
    nodes[k]["connection"]=v

In [26]:
"""Step 5: Determine possible combinations of connections (0,1,2) in a list composed of 4 numbers"""

# Create a list of types having the info of connections using the create_combination definition 
# which iterates every possible combination of 4 numbers and gathers them in a list

# Connection conditions:
type_10 = create_combinations('0000')
type_20 = create_combinations('1000')
type_21 = create_combinations('2000')
type_30 = create_combinations('1100')
type_31 = create_combinations('1200')
type_32 = create_combinations('2200')
type_40 = create_combinations('1110')
type_41 = create_combinations('1120')
type_42 = create_combinations('1210')
type_43 = create_combinations('2210')
type_44 = create_combinations('2120')
type_45 = create_combinations('2220')
type_50 = create_combinations('1100')
type_51 = create_combinations('2001')
type_52 = create_combinations('2200')


"""Step 5.1: Exceptional Modules"""
# Create a seperate dictionary holding the module type name 
# and possible combinations of open/closed 
# for the exceptional modules which should be assigned seperately 
# since the number of closed and open edges are the same for modules 30&50 , modules 41&42 and modules 43&44

# Module types 30 and 50 [0,1,0,1]
types = {30.1: type_30, 31.1: type_31, 32.1:type_32}
counter_types = {30.1: 50.1, 31.1: 51.1, 32.1:52.1}

# Module types 41 and 42 [1,0,1,2]
types_41 = {41.1: type_41}
counter_types_41 = {41.1: 42.1}

# Module types 43 and 44 [2,0,2,1]
types_43 = {43.1: type_43}
counter_types_43 = {43.1: 44.1}

In [27]:
"""Step 6: Introduce a new attribute "module" to nodes dictionary"""

# assign a new empty attribute,module type, to the nodes
for k in nodes:
    nodes[k]["module"]= None

In [28]:
"""Step 6.1: Exception for module families 30&50"""

# Create a dictionary containing closed edges
edge_dict_closed={}
for k,v in edge_dict_connection.items(): 
    for i in v:
        # For module types 30 and 50, there are two 0s and two 1s or 2s in the list
        # therefore we can check the count of 0s instead of checking 1s or 2s
        # if there are two 0s then, these nodes are added to the dictionary we will work with on the next steps 
        closed_edge_count=v.count(0)
        if closed_edge_count == 2:
            edge_dict_closed[k]=edge_dict[k] 

# From this dictionary create a new dictionary 
# holding only the nodes which have closed connection(1 or 2) to the key node
closed_edges={}
for k,v in edge_dict_closed.items():
    closed_nodes=[]
    for no, m in enumerate(edge_dict_connection[k]):
        # print(no)
        if (m==1) or (m==2):
            closed_nodes.append(edge_dict_closed[k][no])
    closed_edges[k]=closed_nodes

# Type 30 and type 50 have an exception as they both have the same list of numbers
# but the position of open&closed edges differ between these two types
# start the if statement by asking whether the closed edge neighbors of the node(which we are assigning a module) have a neighbor in common
# if the nodes that have a closed(1) connection with the parent node have a neighbor in common module type is from type 30 family
# if it is not a neighbor then it should be a module from type 50 family
# Check also for some other conditions which may apply to type 30 and 50 while doing the neighbor check

for k,v in closed_edges.items():
    if (nodes[k]["type"]=="1") or (nodes[k]["type"]=="3") or (nodes[k]["type"]=="2") and (nodes[k]["type"]!=None):
        neigh_neigh = []
        if len(v) < 2:
            continue

        # Find the neighbors of the nodes connected to the node we are assigning a module to
        # Put them into a set
        for i in v:
            neigh_neigh.append(set(G.neighbors(i)))
        nn0, nn1 = tuple(neigh_neigh)

        # Check for common nodes in nn0 and nn1
        intersection = nn0.intersection(nn1)
        for ti, tc in types.items():
            if (nodes[k]["connection"] in tc):
                # Check for 2 intersections because first intersection is the node we are trying to define the module of
                # second intersection is the neighbour of the neighbouring node to the node we are trying to define the module of
                if len(intersection)==2:
                    nodes[k]["module"] = ti 
                else:
                    # If there are no common neighbors of neighbors then closed edges are on the opposite sides (type 50)
                    nodes[k]["module"] = counter_types[ti]         

In [29]:
"""Step 6.2: Exception for modules 41&42"""

# Make a list of keys of exceptional "connection" attributes which would correspond to either module type 41 or 42 [2,1,0,1]
module_41_42_list=[]
for k in nodes:
    if nodes[k]["connection"] in type_41:
       module_41_42_list.append(k)
       
# Create a dictionary holding the current node as key and closed edges(1) as values
reinforced_dict_41={}
for i in module_41_42_list:
    for k in edge_dict_connection:
        reinforced_list_41=[]
        for no,t in enumerate(edge_dict_connection[i]):
             if t==1:
                reinforced_list_41.append(edge_dict[i][no])  
                reinforced_dict_41[i] = reinforced_list_41

# Check if the current node is a shop,workshop or both since we are assigning modules to these functions
for k,v in reinforced_dict_41.items():
    if (nodes[k]["type"]=="1") or (nodes[k]["type"]=="3") or (nodes[k]["type"]=="2") and (nodes[k]["type"]!=None):
        neigh_neigh = []
        if len(v) < 2:
            continue

        # Find the neighbors of the nodes connected to the node we are assigning a module to
        # Put them into a set
        for i in v:
            neigh_neigh.append(set(G.neighbors(i)))
        nn0, nn1 = tuple(neigh_neigh)

        # Check for common nodes in nn0 and nn1
        intersection = nn0.intersection(nn1)
        for ti, tc in types_41.items():
            if (nodes[k]["connection"] in tc):
                # Check for 2 intersections because first intersection is the node we are trying to define the module of
                # second intersection is the neighbour of the neighbouring node to the node we are trying to define the module of
                if len(intersection)==2:
                    nodes[k]["module"] = ti 
                else:
                    # If there are no common neighbors of neighbors then closed edges are on the opposite sides (type 43)
                    nodes[k]["module"] = counter_types_41[ti] 

In [30]:
"""Step 6.3: Exception for modules 41&42"""

# Make a list of keys of exceptional "connection" attributes which would correspond to either module type 43 or 44 [2,2,0,1]
module_43_44_list=[]
for k in nodes:
    if nodes[k]["connection"] in type_43:
       module_43_44_list.append(k)

# Create a dictionary holding the current node as key and reinforced/buttressed edges(2) as values
reinforced_dict_43={}
for i in module_43_44_list:
    for k in edge_dict_connection:
        reinforced_list_43=[]
        for no,t in enumerate(edge_dict_connection[i]):
             if t==2:
                reinforced_list_43.append(edge_dict[i][no])  
                reinforced_dict_43[i] = reinforced_list_43

# Check if the current node is a shop,workshop or both since we are assigning modules to these functions
for k,v in reinforced_dict_43.items():
    if (nodes[k]["type"]=="1") or (nodes[k]["type"]=="3") or (nodes[k]["type"]=="2") and (nodes[k]["type"]!=None):
        neigh_neigh = []
        if len(v) < 2:
            continue

        # Find the neighbors of the nodes connected to the node we are assigning a module to
        # Put them into a set
        for i in v:
            neigh_neigh.append(set(G.neighbors(i)))
        nn0, nn1 = tuple(neigh_neigh)

        # Check for common nodes in nn0 and nn1
        intersection = nn0.intersection(nn1)
        for ti, tc in types_43.items():
            if (nodes[k]["connection"] in tc):
                # Check for 2 intersections because first intersection is the node we are trying to define the module of
                # second intersection is the neighbour of the neighbouring node to the node we are trying to define the module of
                if len(intersection)==2:
                    nodes[k]["module"] = ti 
                else:
                    # If there are no common neighbors of neighbors then closed edges are on the opposite sides (type 44)
                    nodes[k]["module"] = counter_types_43[ti] 

In [31]:
"""Step 6.4: Assign module types to the remaining shop/workshop/shop&workshop nodes"""

# Assigns module types to nodes according to connections except the exceptions we have already made above
for i in nodes:
    if ((nodes[i]["type"]=="1") or (nodes[i]["type"]=="3") or (nodes[i]["type"]=="2")) and (nodes[i]["type"]!=None):
        if (nodes[i]["connection"] in type_10 ) :
            nodes[i]["module"] = 10.1
        elif (nodes[i]["connection"] in type_20) :
            nodes[i]["module"] = 20.1
        elif (nodes[i]["connection"] in type_21) :
            nodes[i]["module"] = 21.1
        elif (nodes[i]["connection"] in type_40) :
            nodes[i]["module"] = 40.1
        elif (nodes[i]["connection"] in type_45) :
            nodes[i]["module"] = 45.1

In [32]:
"""Step 6.5: Switch to filled Module types if necessary"""

# Until now all module types are assigned assuming there is no module above
# If there is a second floor above ("floor"=1) then change the module type to filled (_.2)
# example: (node 0 has a type 20.1 module) and (and there will be a module on it) then (node 0's type = 20.1 + 0.1=20.2) 
# If there is no second floor leave it unfilled (_.1)
for k,i in nodes.items():
    if k < nodes_count_1:
        if nodes[k]["module"]!=None:
            if (nodes[k]["floor"]==1) or nodes[(k+nodes_count_1)]["tag"]==4:
                module=nodes[k]["module"]
                module=round((module+0.1),1)
                nodes[k]["module"]=module

In [33]:
"""Step 7: Assigning modules to local street nodes"""

# Find the local street nodes that have 2 shops next to it (Cornerstone)
# Make a list of local street nodes
covered_street = []
for k in range(len(nodes)):
    if (nodes[k]["tag"] == 0) and (nodes[k]["use_frequency"]=="D"):
       covered_street.append(k) 

# Form a dictionary of the street nodes that have connection to shops and 
# the nodes they are connected to

# Make a list of nodes as keys which are connected to street nodes
edge_dict_copy=edge_dict.copy()
x=[]
for k in covered_street:
    for i in edge_dict_copy:
        if k==i:
            x.append(i)

#  Create a new dictionary holding the local street nodes as keys and their non street neighbors as values
street_dict={}
for k in edge_dict_copy:
    for i in x:
        if k==i:
            street_dict[k]=edge_dict_copy[k]

streetless_dict=street_dict.copy()
for k,v in streetless_dict.items():
    for i in v:
        if (nodes[i]["tag"]==0) or (nodes[i]["tag"]==1) or (nodes[i]["tag"]==2):
            v.remove(i)

# Assign module type 10.1 to the nodes which have shops on the sides
for k,v in streetless_dict.items():
    for count,item in enumerate(v):
        if (nodes[v[count]]["type"]==nodes[v[(count+1)%len(v)]]["type"]) and (nodes[item]["type"]=="1") and (nodes[item]["floor"]==1):
            nodes[k]["module"] = 10.1 

In [34]:
"""Step 7.1: Changing module types to unfilled (_.1) from filled(_.2)"""

# If the neighbour of the walkpath is a shop/workshop/both node below the walkpath should be assigned a module which is filled(_.2)
# Otherwise an unfilled module (_.1)
for k,v in edge_dict.items():
    if k < nodes_count_1:
        for i in v:
            m = 1
            # Check if it is an occupied node and there a module above that node
            if (nodes[k]["module"])!= None and nodes[k]["floor"] == 1:
                upper_level=k+nodes_count_1

                for t in edge_dict[upper_level]:
                    #if neighbour's tag above me is everything but a name (a.k.a a shop/workshop/both), make m = 1, to subtract 0.1 to make it an unwalkable path
                    no_shop_condition = [None, "x", 0, 1, 2, 3, 4]
                    if nodes[t]["tag"] in no_shop_condition:
                        if nodes[upper_level]["tag"]==4:
                            m *= 1
                        else:
                            m *= 0
                    else:
                        m *= 0                   
                # Subtract only when there is no shop/workshop/both above
                module=(nodes[k]["module"] - 0.1 * m)
                module=round(module,1)
                nodes[k]["module"]=module
                break               

In [35]:
"""Step 7.2: Assign modules to walkpaths"""

# Make a list of filled module types
covered_second_floor_modules= [10.2, 20.2, 21.2, 30.2, 31.2, 32.2, 40.2, 41.2, 42.2, 43.2, 44.2, 45.2, 50.2]
for k,v in nodes.items():
    if k<nodes_count_1:
        # If the node above is a walkpath and it is not assigned a module and the node below has a filled module
        if (nodes[k+nodes_count_1]["tag"]==4) and (nodes[k+nodes_count_1]["module"]==None) and (nodes[k]["module"] in covered_second_floor_modules):
            # Then assign an all sides open module type to the node above
                nodes[k+nodes_count_1]["module"]=10.1

In [36]:
"""Step 8: Exporting final nodes attributes and edge_dict used for orientation in rhino"""

file_nodes = "nodes_final.txt"
file_edges = "edge_dict.txt"

with open(file_nodes,"w") as nodes_outfile:
    try:
        json.dump(nodes, nodes_outfile)
        print(file_nodes + " has been updated successfully")
    except:
        print("Problem with updating file: ", file_nodes)

with open(file_edges,"w") as edges_outfile:
    try:
        json.dump(edge_dict, edges_outfile)
        print(file_edges + " has been updated successfully")
    except:
        print("Problem with updating file: ", file_edges)

nodes_final.txt has been updated successfully
edge_dict.txt has been updated successfully
